In [1]:
import django, sys, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
from scipy.sparse import coo_matrix, find
from sklearn.manifold import TSNE

from scoping.models import *
from tmv_app.models import *

from scipy.spatial import ConvexHull

run_id = 662
s = 0
p = 200

dto = DocDynamicTopic.objects

m = np.load('../tsne_results/data/run_{}_s_{}_m.npy'.format(run_id,s))
r_ind = np.load('../tsne_results/data/run_{}_s_{}_r_ind.npy'.format(run_id,s))
tsne_results = np.load('../tsne_results/data/run_{}_s_{}_p{}.npy'.format(run_id,s,p))

In [2]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.18,min_samples=50).fit(tsne_results)

labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)


print('Estimated number of clusters: %d' % n_clusters_)

Estimated number of clusters: 119


In [3]:
label_set = list(set(labels))
for l in set(labels):
    ind = np.argwhere(labels==l)
    print("\n##\nlabel: {}, {} documents".format(l,len(ind)))
    


##
label: 0, 1061 documents

##
label: 1, 1314 documents

##
label: 2, 1301 documents

##
label: 3, 9784 documents

##
label: 4, 6878 documents

##
label: 5, 10397 documents

##
label: 6, 14063 documents

##
label: 7, 1072 documents

##
label: 8, 10405 documents

##
label: 9, 2305 documents

##
label: 10, 804 documents

##
label: 11, 2031 documents

##
label: 12, 2034 documents

##
label: 13, 2438 documents

##
label: 14, 9721 documents

##
label: 15, 7655 documents

##
label: 16, 1926 documents

##
label: 17, 1464 documents

##
label: 18, 299 documents

##
label: 19, 1413 documents

##
label: 20, 1343 documents

##
label: 21, 3227 documents

##
label: 22, 2103 documents

##
label: 23, 9145 documents

##
label: 24, 3617 documents

##
label: 25, 933 documents

##
label: 26, 3198 documents

##
label: 27, 204 documents

##
label: 28, 4136 documents

##
label: 29, 1422 documents

##
label: 30, 4267 documents

##
label: 31, 3253 documents

##
label: 32, 3783 documents

##
label: 33, 2685 d

In [ ]:
label_set = list(set(labels))

fig = plt.figure(dpi=188)


import matplotlib.colors as colors
import matplotlib.cm as cmx
jet = cm = plt.get_cmap('jet') 
cNorm  = colors.Normalize(vmin=min(label_set), vmax=max(label_set))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

df = pd.DataFrame()

doc_df = pd.DataFrame()

colors = ['#7fc97f','#beaed4','#fdc086','#ffff99','#386cb0','#f0027f','#F0F0F0']
oecd_cats = list(Doc.objects.filter(id__in=r_ind).distinct('wc__oecd').values('wc__oecd'))
for i,c in enumerate(oecd_cats):
    c['docs'] = list(Doc.objects.filter(id__in=r_ind,wc__oecd=c['wc__oecd']).values_list('id',flat=True))
    c['color'] = colors[i]

for l in list(set(labels)):
    ind = np.argwhere(labels==l)
    cs = []
    sizes = []
    xs = []
    ys = []
    
    dids = []

    for i,did in enumerate(ind):
        x = tsne_results[did[0],0]
        y = tsne_results[did[0],1]
        col = "grey"
        for c in oecd_cats:
            if r_ind[did[0]] in c['docs']:
                col = c['color']
        cs.append(col)
        xs.append(x)
        ys.append(y)
        dids.append(r_ind[did[0]])
        
    cdocs = Doc.objects.filter(id__in=dids).values('title','PY')
    rows = pd.DataFrame.from_dict(list(cdocs))
    rows['cluster'] = l
    doc_df = doc_df.append(rows)
                
    if l !=-1:
        points = np.array([(xs[i],ys[i]) for i,v in enumerate(xs)])
        hull = ConvexHull(points)
        for i, simplex in enumerate(hull.simplices):
            plt.plot(
                points[simplex, 0], 
                points[simplex, 1], 
                'k-',
                linewidth=0.5
            )
            for j in range(len(points[simplex,0])):                
                if i==0 and j==0:
                    px = points[simplex,0][j]
                    py = points[simplex,1][j]
                else:
                    if points[simplex,0][j] < px:
                        px = points[simplex,0][j]
                        py = points[simplex,1][j]                    

        plt.text(
            px-1,
            py,
            l,
            va="center",
            ha="right",
            fontsize=5
        )
        
    plt.scatter(
        xs,
        ys,
        s=3,
        #s=sizes,
        alpha=0.6,
        linewidth=0.1,
        c=cs,
        edgecolor='k'
    )
    
l = plt.xlim()[0]
t = plt.ylim()[1]
yextent = plt.ylim()[1]- plt.ylim()[0]
ysp = yextent*0.08

for i,c in enumerate(oecd_cats):
    plt.text(
        l*0.95,
        t-ysp-i*ysp,
        "{} {:.1%}".format(c['wc__oecd'],len(c['docs'])/len(r_ind)),
        fontsize=4,
        bbox={
            'facecolor': c['color'],
            'pad': 2,
            'lw': 0.5
        }
    )

plt.tick_params(
    axis='both',          
    which='both',      
    bottom=False,      
    top=False,  
    labelbottom=False,
    left=False,
    labelleft=False
)
            
plt.savefig('../tsne_results/plots/run_{}_s_{}_p{}_clusters.png'.format(run_id,s,p))
plt.show()

In [7]:
df.to_csv('../tsne_results/data/run_{}_s_{}_p{}_clusters.png'.format(run_id,s,p))

In [ ]:
label_set = list(set(labels))

fig = plt.figure(dpi=188)

colors=["#66c2a5","#fc8d62","#8da0cb"]



wgs = list(scoping.models.WG.objects.all().distinct('wg').values('wg'))
for i,c in enumerate(wgs):
    c['docs'] = list(Doc.objects.filter(
        id__in=r_ind,ipccref__wg__wg=c['wg']
    ).values_list('id',flat=True))
    c['color'] = colors[i]

for l in list(set(labels)):
    ind = np.argwhere(labels==l)
    cs = []
    sizes = []
    xs = []
    ys = []
    alphas=[]
    
    dids = []

    for i,did in enumerate(ind):
        x = tsne_results[did[0],0]
        y = tsne_results[did[0],1]
        col = "#F0F0F026"
        al = 0.5
        for c in wgs:
            if r_ind[did[0]] in c['docs']:
                col = c['color']
                al = 1
        alphas.append(al)
        cs.append(col)
        xs.append(x)
        ys.append(y)
        dids.append(r_ind[did[0]])
        
    cdocs = Doc.objects.filter(id__in=dids).values('title','PY')
    rows = pd.DataFrame.from_dict(list(cdocs))
    rows['cluster'] = l
    doc_df = doc_df.append(rows)
        
    if l !=-1:
        points = np.array([(xs[i],ys[i]) for i,v in enumerate(xs)])
        hull = ConvexHull(points)
        for i, simplex in enumerate(hull.simplices):
            plt.plot(
                points[simplex, 0], 
                points[simplex, 1], 
                'k-',
                linewidth=0.5
            )
            for j in range(len(points[simplex,0])):                
                if i==0 and j==0:
                    px = points[simplex,0][j]
                    py = points[simplex,1][j]
                else:
                    if points[simplex,0][j] < px:
                        px = points[simplex,0][j]
                        py = points[simplex,1][j]                    

        plt.text(
            px-1,
            py,
            l,
            va="center",
            ha="right",
            fontsize=5
        )
        
    plt.scatter(
        xs,
        ys,
        s=3,
        #alpha=alphas,
        #s=sizes,
        linewidth=0.1,
        c=cs,
        edgecolor='#a39c9c99'
    )
    
l = plt.xlim()[0]
t = plt.ylim()[1]
yextent = plt.ylim()[1]- plt.ylim()[0]
ysp = yextent*0.08

for i,c in enumerate(wgs):
    plt.text(
        l*0.95,
        t-ysp-i*ysp,
        "WG {} {:.1%}".format(c['wg'],len(c['docs'])/len(r_ind)),
        fontsize=4,
        bbox={
            'facecolor': c['color'],
            'pad': 2,
            'lw': 0.5
        }
    )

plt.tick_params(
    axis='both',          
    which='both',      
    bottom=False,      
    top=False,  
    labelbottom=False,
    left=False,
    labelleft=False
)
            
plt.savefig('../tsne_results/plots/run_{}_s_{}_p{}_clusters_wgs.png'.format(run_id,s,p))
plt.show()